In [48]:
import httpx
from telegram import Update, ext
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

# Замените на ваши значения
TELEGRAM_TOKEN = 'токен_выслал_в_лмс'
HUGGINGFACE_API_KEY = 'токен_выслал_в_лмс'
MODEL_NAME = 'google/gemma-7b' # плюс-минус работающая модель, но если ей писать на русском происходит взрыв градиента)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Отправляет сообщение при команде /start."""
    await update.message.reply_text('Привет! Я чат бот, который ответит на все интересующие тебя вопросы, но только на английском (на русском мне плохо :))')

# Функция для генирации ответа LLM, он находится на бесплатном ресурсе, поэтому сильно глупее 3.5 GPT, тем более 4
async def generate_text(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Генерирует текст на основе ввода пользователя с использованием LLM от Hugging Face."""
    prompt_text = update.message.text

    headers = {
        'Authorization': f'Bearer {HUGGINGFACE_API_KEY}'
    }
    payload = {
        'inputs': prompt_text,
        'parameters': {'max_length': 100},
        'options': {'use_cache': False}
    }

    async with httpx.AsyncClient() as client:
        response = await client.post(f'https://api-inference.huggingface.co/models/{MODEL_NAME}', json=payload, headers=headers)
    
    # Разные хендлери, ожидает результата и возвращает ответ
    if response.status_code == 200:
        generated_text = response.json()[0]['generated_text']
        await update.message.reply_text(generated_text)
    else:
        await update.message.reply_text('Произошла ошибка при генерации текста.')

def main():
    application = Application.builder().token(TELEGRAM_TOKEN).build()
    
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate_text))

    application.run_polling()

if __name__ == '__main__':
    main()

RuntimeError: Cannot close a running event loop